In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.stats import mode
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, plot_confusion_matrix, roc_curve, precision_recall_curve
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_part1 = pd.read_csv("Part 1.tsv", sep="\t")
X = df_part1.drop("label", axis=1).values
y = df_part1["label"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

In [ ]:
class Neural_Network(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.name = "Neural Network"
        self.learning_rate = 0.01
        self.costs = []
        self.accuracies = []
        self.classes_ = [0,1]

    def _sigmoid(self, Z:np.ndarray):
        return 1.0 / (1.0 + np.exp(-Z))

    def _sigmoid_derivative(self, output:np.ndarray):
        return output * (1.0 - output)

    def _cost_function(self, y:np.ndarray, y_hat:np.ndarray):
        return -np.sum(y*(np.log(y_hat)) + (1 - y)*np.log(1 - y_hat)) / y.shape[0]

    def _cost_gradient(self, y:np.ndarray, y_hat:np.ndarray):
        return -np.divide(y, y_hat) + np.divide(1.0 - y, 1.0 - y_hat)
    
    def _tanh(self, Z:np.ndarray) -> np.ndarray:
        return np.tanh(Z)

    def _sech(self, Z:np.ndarray) -> np.ndarray:
        return 1/np.cosh(Z)

    def _tanh_derivative(self,Z:np.ndarray) -> np.ndarray:
        return self._sech(Z)**2

    def forward_pass(self, X_train_1:np.ndarray, X_train_2:np.ndarray):
        
        self.A0 = np.hstack([X_train_1,np.ones( (X_train_1.shape[0],1) )] ).T

        self.Z1 = np.dot(self.W1, self.A0)

        self.A1 = self._tanh(self.Z1)
        self.A1 = np.concatenate((self.A1, X_train_2.T), axis=0)

        self.A1 = np.vstack([self.A1, np.ones((1, self.A1.shape[1] ))])
        self.Z2 = np.dot(self.W2, self.A1)

        self.A2 = self._sigmoid(self.Z2)

        # self.accuracies.append(np.round(self.A))
        self.accuracies.append(np.mean(np.round(self.A2)==y_train))

        
    def backward_pass(self, y_train:np.ndarray):
        self.dA2 = self._cost_gradient(y_train, self.A2)
        self.dZ2 = self.dA2*self._sigmoid_derivative(self._sigmoid(self.Z2))

        self.m1 = self.A1.shape[1]
        self.dW2 = np.dot(self.dZ2, self.A1.T)/self.m1
        
        self.dA1 = np.dot(self.W2[:,:1].T, self.dZ2)

        self.dZ1 = self.dA1*self._tanh_derivative(self._tanh(self.Z1))

        self.m0 = self.A0.shape[1]
        self.dW1 = self.dZ1 @ self.A0.T/self.m0

        self.W1 -= self.dW1*self.learning_rate
        self.W2 -= self.dW2*self.learning_rate

        self.costs.append(self._cost_function(y_train, self.A2))


        
    def fit(self, X_train:np.ndarray, y_train:np.ndarray, epochs=1000):
        # Dessa maneira, o modelo funciona com qualquer dataset
        # E, também, funciona com o atual dataset obedecendo a forma pedida pelo trabalho
        self.input_hidden_1 = X_train.shape[1]//2
        self.input_hidden_2 = X_train.shape[1]//2 + X_train.shape[1]%2 + 1
        
        self.W1 = np.random.random((1, self.input_hidden_1+1))
        self.W2 = np.random.random((1, self.input_hidden_2+1))
        
        self.epochs = epochs

        X_train_1 = X_train[:, 0:self.input_hidden_1]
        X_train_2 = X_train[:, self.input_hidden_1:]
        
        for e in range(self.epochs):
            self.forward_pass(X_train_1, X_train_2)
            self.backward_pass(y_train)
        return self
    
    def predict_proba(self, X_test:np.ndarray):
        X_test_1 = X_test[:, 0:self.input_hidden_1]
        X_test_2 = X_test[:, self.input_hidden_1:]
        A0 = np.hstack([X_test_1, np.ones( (X_test_1.shape[0],1) )] ).T
        
        Z1 = np.dot(self.W1, A0)
        A1 = self._tanh(Z1)
        A1 = np.concatenate((A1, X_test_2.T), axis=0)

        A1 = np.vstack([A1, np.ones((1, A1.shape[1] ))])
        
        Z2 = np.dot(self.W2, A1)

        A2 = self._sigmoid(Z2)
        
        return A2[0]
    
    def predict(self, X_test:np.ndarray):
        return np.round(self.predict_proba(X_test))


In [ ]:
NN = Neural_Network()
NN.fit(X_train, y_train, 5000)

In [ ]:
report = pd.DataFrame(classification_report(y_test, NN.predict(X_test), output_dict=True))
metrics = {}
metrics["Accuracy"] = [report["accuracy"].mean(axis=0)]
metrics["Precision"] = [report.mean(axis=1)["precision"]]
metrics["Recall"] = [report.mean(axis=1)["recall"]]
metrics["AUC-ROC"] = [roc_auc_score(y_test, NN.predict_proba(X_test))]
metrics["AUC-PR"] = [average_precision_score(y_test, NN.predict_proba(X_test))]

table_metrics = pd.DataFrame(metrics)
table_metrics

In [ ]:
plot_confusion_matrix(NN,X_test, y_test)
plt.show()

In [ ]:
# Colocar título e legenda
fig, ax = plt.subplots()

sns.lineplot(x=range(len(NN.costs)), y=NN.costs, label="Cost Error", ax=ax)
sns.lineplot(x=range(len(NN.accuracies)), y=NN.accuracies, label="Trainning Accuracy", ax=ax)
ax.set_title("Neural Network Error")
ax.set_xlabel("Epochs")
plt.show()